In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import timm
import torch

from augmentations import Masking, GaussianNoise, Stretch
from data import get_datasets
from AdaMatch import AdaMatch
from Baseline import Baseline
from FixMatch import FixMatch
from model import Model
from metrics import AUC, Error_Rate

In [2]:
# Hyperparameters
epochs = 2048
learning_rate = 1e-3
momentum = 0.9
mu = 7
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Data
train_dataset, unlabeled_dataset, test_dataset = get_datasets('./data', name='mnist', download=True, transform=transforms.ToTensor(), n_labeled=24, seed=4)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=batch_size*mu, shuffle=True)


In [3]:
np.unique([np.argmax(train_dataset[i][1]) for i in range(20)])

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [4]:
weak = transforms.Compose([
    transforms.RandomAffine(45, translate=(.1, .1)),
])

strong = transforms.Compose([
    transforms.RandomAutocontrast(),
    transforms.GaussianBlur(kernel_size=(5,9), sigma=(.1, 2.)),
    transforms.RandomInvert(),
    transforms.RandomAffine(90, translate=(.3, .3), scale=(.8, 1.2), shear=(-.1, .1, -.1, .1)),
    transforms.RandomErasing()
])

In [5]:
config = {'backbone':'resnet18','n_channels':1}
model = Model(10, config=config)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs, eta_min=1e-8)
criterion = nn.functional.cross_entropy
metrics = [AUC,Error_Rate]

trainer = Baseline(weak_transform=weak)
model, log_baseline = trainer.train(train_loader, test_loader, model, optimizer, criterion, metrics, scheduler=scheduler, epochs=epochs)

Epoch 2048/2048
 Training 	|	 loss=0.023
 Validation 	|	 loss=200.938  -  AUC=0.937  -  Error_Rate=0.37

 Best : AUC=0.936  -  Error_Rate=0.369 at epoch 500


In [ ]:
config = {'backbone':'resnet18', 'n_channels':1}
model = Model(10, config=config)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs, eta_min=1e-8)
criterion = nn.functional.cross_entropy
metrics = [AUC,Error_Rate]


trainer = FixMatch(tau=0.95, strong_transform=strong, weak_transform=weak)
model, log_fixmatch = trainer.train(train_loader, unlabeled_loader, test_loader, model, optimizer, criterion, metrics, scheduler=scheduler, epochs=epochs)

Epoch 1937/2048
 Training 	|	 loss=[0.291 0.012 0.28 ]  |  111.0 Pseudo-labels
 Validation 	|	 loss=160.676  -  AUC=0.941  -  Error_Rate=0.304

 Best : AUC=0.945  -  Error_Rate=0.296 at epoch 1072
